# **Launch Sites Locations Analysis with Folium**
IMB's Applied Data Science Capstone Project

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives
- create interactive map  with Folium
- find the geoghraphical factors that the could affect the launch success rate 


In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

##  Mark all launch sites on a map


In [3]:
# Download and read the `spacex_launch_geo.csv`
spacex_df=pd.read_csv('data/Falcon_dataset.csv')
pd.set_option('display.max_columns', None)

In [4]:
spacex_df

,FlightNumber,Date,BoosterVersion,LaunchSite,Payload,PayloadMass,Orbit,Customer,LaunchOutcome,LandingOutcome,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,F9 v1.0,CCSFS SLC 40,Dragon Spacecraft Qualification Unit,6124,LEO,SpaceX,Success,Failure,None None,1,False,False,False,NaN,1,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,F9 v1.0,CCSFS SLC 40,SpaceX COTS Demo Flight 2,525,LEO,NASA,Success,No attempt,None None,1,False,False,False,NaN,1,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,F9 v1.0,CCSFS SLC 40,SpaceX CRS-2,677,ISS,NASA,Success,No attempt,None None,1,False,False,False,NaN,1,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,F9 v1.1,VAFB SLC 4E,CASSIOPE,500,PO,MDA,Success,Uncontrolled,False Ocean,1,False,False,False,NaN,1,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,F9 v1.1,CCSFS SLC 40,SES-8,3170,GTO,SES,Success,No attempt,None None,1,False,False,False,NaN,1,0,B1004,-80.577366,28.561857,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,86,2020-09-03,F9 B5,KSC LC 39A,Starlink,15600,VLEO,SpaceX,Success,Success,True ASDS,2,True,True,True,5e9e3032383ecb6bb234e7ca,5,12,B1060,-80.603956,28.608058,1
85,87,2020-10-06,F9 B5,KSC LC 39A,Starlink,15600,VLEO,SpaceX,Success,Success,True ASDS,3,True,True,True,5e9e3032383ecb6bb234e7ca,5,13,B1058,-80.603956,28.608058,1
86,88,2020-10-18,F9 B5,KSC LC 39A,Starlink,15600,VLEO,SpaceX,Success,Success,True ASDS,6,True,True,True,5e9e3032383ecb6bb234e7ca,5,12,B1051,-80.603956,28.608058,1
87,89,2020-10-24,F9 B5,CCSFS SLC 40,Starlink,15600,VLEO,SpaceX,Success,Success,True ASDS,3,True,True,True,5e9e3033383ecbb9e534e7cc,5,12,B1060,-80.577366,28.561857,1


Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Latitude`, `Longitude`, `class`
spacex_df = spacex_df[['LaunchSite', 'Latitude', 'Longitude', 'Class']]
launch_sites_df = spacex_df.groupby(['LaunchSite'], as_index=False).first()
launch_sites_df = launch_sites_df[['LaunchSite', 'Latitude', 'Longitude']]
launch_sites_df['launch_site_coordinate'] = launch_sites_df.apply(lambda row: [row['Latitude'], row['Longitude']], axis=1)
launch_sites_df.drop(['Latitude', 'Longitude'], axis=1, inplace=True)
launch_sites_df

,LaunchSite,launch_site_coordinate
0,CCSFS SLC 40,"[28.5618571, -80.577366]"
1,KSC LC 39A,"[28.6080585, -80.6039558]"
2,VAFB SLC 4E,"[34.632093, -120.610829]"


In [6]:
#turn launch_sites_df to dict
launch_sites_dict= {key: value for key, value in zip(launch_sites_df['LaunchSite'], launch_sites_df['launch_site_coordinate'])}

## visualize those locations by pinning them on a map


- create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.
- use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10, width=1000, height=600)

In [8]:
fmap = folium.Map(location=[33.116779, -100.176059], zoom_start=5, width=1000, height=600)
for launch_site, coordinates in launch_sites_dict.items():
    launch_site_name = launch_site
    coordinates = coordinates
    c = folium.Circle(coordinates, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(launch_site_name))
    m = folium.map.Marker(coordinates, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_site_name, ))
    fmap.add_child(c)
    fmap.add_child(m)
fmap

- explore the map by zoom-in/out the marked areas
- we can see that all launch sites are close to Equator line and the coast

## Mark the success/failed launches for each site on the map


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


In [9]:
#create a `MarkerCluster` object
marker_cluster = MarkerCluster()

In [10]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['Class'].apply(assign_marker_color)
spacex_df.tail(10)

/var/folders/55/d9syjzsd0w1dw_5pcrb19hg80000gn/T/ipykernel_4130/4141172157.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spacex_df['marker_color'] = spacex_df['Class'].apply(assign_marker_color)


,LaunchSite,Latitude,Longitude,Class,marker_color
79,CCSFS SLC 40,28.561857,-80.577366,1,green
80,CCSFS SLC 40,28.561857,-80.577366,1,green
81,CCSFS SLC 40,28.561857,-80.577366,1,green
82,CCSFS SLC 40,28.561857,-80.577366,1,green
83,CCSFS SLC 40,28.561857,-80.577366,1,green
84,KSC LC 39A,28.608058,-80.603956,1,green
85,KSC LC 39A,28.608058,-80.603956,1,green
86,KSC LC 39A,28.608058,-80.603956,1,green
87,CCSFS SLC 40,28.561857,-80.577366,1,green
88,CCSFS SLC 40,28.561857,-80.577366,1,green


In [11]:
# Add marker_cluster to current site_map
fmap.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    launch_site_coordinates = [record['Latitude'], record['Longitude']]
    #Create and add a Marker cluster to the site map
    marker = folium.Marker(
        location=launch_site_coordinates,
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        tooltip=f"Launch Outcome: {record['Class']}"
    )
    marker_cluster.add_child(marker)


    
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

fmap.add_child(mouse_position)
fmap

In [12]:
# Iterate through the DataFrame and add markers to the cluster
for index, record in spacex_df.iterrows():
    launch_site_coordinates = [record['Latitude'], record['Longitude']]
    marker_color = record['marker_color']
    
    marker = folium.Marker(
        location=launch_site_coordinates,
        icon=folium.Icon(color=marker_color),
        tooltip=f"Launch Outcome: {record['Class']}"
    )
    marker_cluster.add_child(marker)

# Display the map
fmap.save('site_map.html')  # Save the map to an HTML file


## Calculate the distances between a launch site to its proximities


In [13]:
#calculate the distance between two points on the map based on their `Lat` and `Long` values
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [15]:
import folium
from folium.features import DivIcon

def create_map_with_markers(launch_site_coordinate, locations):

    
    # Create a folium map
    fmap = folium.Map(location=[28.612, -81.138], zoom_start=10, width=1000, height=600)
    
    launch_marker = folium.Marker(
    location=launch_site_coordinate,
    icon=folium.Icon(color='red', prefix='fa')
    )
    # Create a MousePosition control to display coordinates
    mouse_position = folium.plugins.MousePosition()
    mouse_position.add_to(fmap)
    
    for location in locations:
        # Calculate the midpoint between launch site and coastline
        mid_latitude = (launch_site_coordinate[0] + location["latitude"]) / 2
        mid_longitude = (launch_site_coordinate[1] + location["longitude"]) / 2
        # Calculate distance between launch site and location
        distance_location = calculate_distance(
            launch_site_coordinate[0], launch_site_coordinate[1],
            location["latitude"], location["longitude"]
        )
        
        # Create markers for location and distance
        location_marker = folium.Marker(
            location=[location["latitude"], location["longitude"]],
            popup=location["name"]
        )
        distance_marker = folium.Marker(
            location=[mid_latitude, mid_longitude],
            icon=DivIcon(
                icon_size=(20, 20),
                icon_anchor=(0, 0),
                html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_location),
            )
        )
        
        # Create a polyline from the launch site to the location
        line_coordinates = [
            (location["latitude"], location["longitude"]),
            (launch_site_coordinate[0], launch_site_coordinate[1])
        ]
        polyline = folium.PolyLine(locations=line_coordinates, weight=1)
        
        # Add markers and polyline to the map
        fmap.add_child(location_marker)
        fmap.add_child(distance_marker)
        fmap.add_child(launch_marker)
        fmap.add_child(polyline)
    
    return fmap

# Define the locations and launch site coordinate
locations = [
    {"name": "Closest Coastline", "latitude": 28.610849799026173, "longitude": -80.59656731156174},
    {"name": "Florida East Coast Railway", "latitude": 28.547670, "longitude": -80.797245},
    {"name": "Highway I-95", "latitude": 28.548577335896038,"longitude":  -80.85151431465404},
    {"name": "Orlando", "latitude": 28.536, "longitude": -81.382}
]
launch_site_coordinate = [28.6080585, -80.6039558]

# Create the map with markers and polylines
map_with_markers = create_map_with_markers(launch_site_coordinate, locations)
map_with_markers

